In [1]:
!nvidia-smi

Wed Jul  9 09:48:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             26W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:

!pip uninstall -y fsspec
!pip install fsspec==2023.6.0

Found existing installation: fsspec 2023.6.0
Uninstalling fsspec-2023.6.0:
  Successfully uninstalled fsspec-2023.6.0
  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires

In [3]:
from datasets import load_dataset
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [5]:
print(dataset['test']['text'][:10])

['', ' = Robert Boulter = \n', '', ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n', ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill . He

In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token          # GPT‑2 has no PAD
tokenizer.model_max_length = 1024


In [7]:
def tokenize(batch):
    return tokenizer(batch["text"])

tokenized = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing",
)

Tokenizing:   0%|          | 0/4358 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/36718 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
block_size = 512          # 1024 works too if memory allows

def chunk(batch):
    # flatten then chop into block_size pieces
    ids = sum(batch["input_ids"], [])
    total_len = (len(ids) // block_size) * block_size
    ids = ids[:total_len]
    chunks = [ids[i : i + block_size] for i in range(0, total_len, block_size)]
    return {"input_ids": chunks, "labels": chunks.copy()}

lm_ds = tokenized.map(chunk, batched=True, remove_columns=tokenized["train"].column_names)
lm_ds

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 551
    })
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 4656
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 481
    })
})

In [9]:
from transformers import GPT2LMHeadModel

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

batch_size = 2                    # use the largest that fits; gradient_accum >1 if needed
args = TrainingArguments(
    output_dir="gpt2-wikitext2",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=8,        # effective batch = 16
    learning_rate=5e-5,
    num_train_epochs=3,
    do_eval=True,
    save_strategy="epoch",
    fp16=True,                           # automatic mixed precision on Tesla T4/V100/A100
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-10-2893056308.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

output_dir="./wikitext_finetuned"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.252700


('./wikitext_finetuned/tokenizer_config.json',
 './wikitext_finetuned/special_tokens_map.json',
 './wikitext_finetuned/vocab.json',
 './wikitext_finetuned/merges.txt',
 './wikitext_finetuned/added_tokens.json')

In [12]:
import math
eval_results = trainer.evaluate()
print(f"Validation loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity:      {math.exp(eval_results['eval_loss']):.2f}")

Validation loss: 3.1298
Perplexity:      22.87


In [14]:
import torch
from torch.nn.functional import softmax

prompt = "I am your"

# ── Encode prompt ───────────────────────────────
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# ── Safety: set pad_token_id for GPT‑2 family ──
model.config.pad_token_id = tokenizer.eos_token_id

# ── 1️⃣  Greedy continuation (20 new tokens) ────
greedy_out = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=20,         # ← clearer than max_length
    do_sample=False            # greedy
)
print("Greedy continuation:\n", tokenizer.decode(greedy_out[0], skip_special_tokens=True))

# ── 2️⃣  Top‑k next‑token ranking (k=10) ────────
with torch.no_grad():
    logits = model(input_ids=input_ids, attention_mask=attention_mask).logits

next_logits = logits[0, -1]                       # last position
probs = softmax(next_logits, dim=-1)
top_k = probs.topk(10)
tokens = tokenizer.convert_ids_to_tokens(top_k.indices.tolist())
probs  = top_k.values.tolist()

print("\nTop‑10 next‑token probabilities:")
for i, (tok, p) in enumerate(zip(tokens, probs), 1):
    print(f"{i:2d}. {tok!r:15s}  p={p:.4f}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Greedy continuation:
 I am your father . " 
 = = = = = = = = = = = = = = =

Top‑10 next‑token probabilities:
 1. 'Ġfather'        p=0.0717
 2. 'Ġfriend'        p=0.0463
 3. 'Ġmother'        p=0.0339
 4. 'Ġbest'          p=0.0219
 5. 'Ġhusband'       p=0.0206
 6. 'Ġenemy'         p=0.0170
 7. 'Ġbrother'       p=0.0141
 8. 'Ġservant'       p=0.0133
 9. 'Ġgreatest'      p=0.0125
10. 'Ġson'           p=0.0117
